# Group 10: Title 

### Background information:
    The ATP (Association of Tennis Professionals) Tour is a worldwide tennis tour for men’s professional tennis organized annually by the ATP since 1990. The tour is divided into several tiers of tournaments, with higher tiers offering more ranking points. The PIF ATP Rankings is a merit-based method used by the ATP to rank players, determine entry qualification, and organize tournament seeding. Points are awarded based on the stage of the tournament reached and the tier. The rankings are updated weekly and rewarded points are dropped 52 weeks after being awarded, with some exceptions.


### Our research question:
    We plan to predict the best ranking of players based on various predictors. What is a player’s predicted best rank?

### The dataset:
    We will be using a data set containing player stats for the top 500 ATP players from 2017-2019. The dataset contains 37 variables, including both quantitative and qualitative values. 




## Preliminary Exploratory Data Analysis

In [1]:
# Retrieving nessesary packages
library(dplyr)
library(tidyverse)
library(rvest)
library(tidymodels)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.4
✔ ggplot2   3.4.4     ✔ stringr   1.5.0
✔ lubridate 1.9.3     ✔ tibble    3.2.1
✔ purrr     1.0.2     ✔ tidyr     1.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘rvest’


The following object is masked from ‘package:readr’:

    guess_encoding


── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.

In [27]:
# Reading the data file
url = "https://drive.google.com/uc?export=download&id=1_MECmUXZuuILYeEOfonSGqodW6qVdhsS"
player_data = read_csv(url)

# Cleaning the data and getting what we need
processed_player_data = player_data |>
                        separate(Age, c("Age", "Birth date"), sep = " ") |>
                        separate(current_rank, c("Current Rank", "Current Rank (Other)"), sep = " ")

processed_player_data =

ERROR: Error in parse(text = x, srcfile = src): <text>:11:0: unexpected end of input
9: 
10: processed_player_data =
   ^


In [ ]:
# Summary for data
data_summary = data

data_summary

In [ ]:
# Visualization for data
options(repr.plot.width=20, repr.plot.height=20)

data_plot = data |>
            ggplot(aes(x=..,y=..)) +
            geom_point() +
            xlab("x") +
            ylab("y") +
            ggTitle("x vs y") +
            theme(text = element_text(size=20))

data_plot

## Methods

### Relevant variables:
- Predicted Variable: Best Rank
  
Predictors:
- Age
- Current Rank
- Height
- Year Turned pro
- Current ELO Rank
- Best ELO Rank
- Peak ELO Rating
- Weight
- Plays (Left-handed or Right-handed)
- Backhand (Two-Handed or One-Handed)
- Seasons Played
- Country


### Outlined procedure:
1. one
2. two
3. three
4. four
5. five

### Visualization:
We plan to visualize the accuracy vs 


## Expected Outcomes and Significance

### Expected findings:
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nam in sagittis velit. Pellentesque eget pretium ipsum. Proin sed dui neque. Nulla venenatis enim sit amet ipsum consequat, placerat euismod odio mattis. Nunc sed lobortis magna, sed ultricies justo. Etiam aliquet dictum metus non lobortis. Aenean sed suscipit arcu. Sed nunc tellus, condimentum id facilisis elementum, egestas ut libero. Donec sit amet elementum enim, eu luctus mi.v

### Relevancy of findings:
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nam in sagittis velit. Pellentesque eget pretium ipsum. Proin sed dui neque. Nulla venenatis enim sit amet ipsum consequat, placerat euismod odio mattis. Nunc sed lobortis magna, sed ultricies justo. Etiam aliquet dictum metus non lobortis. Aenean sed suscipit arcu. Sed nunc tellus, condimentum id facilisis elementum, egestas ut libero. Donec sit amet elementum enim, eu luctus mi.v

### Future studies:
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nam in sagittis velit. Pellentesque eget pretium ipsum. Proin sed dui neque. Nulla venenatis enim sit amet ipsum consequat, placerat euismod odio mattis. Nunc sed lobortis magna, sed ultricies justo. Etiam aliquet dictum metus non lobortis. Aenean sed suscipit arcu. Sed nunc tellus, condimentum id facilisis elementum, egestas ut libero. Donec sit amet elementum enim, eu luctus mi.v